# Convolutional Neural Network for text classification

Convolutional Neural Network (CNNs) was originally for Computer Vision tasks. [Kim](https://arxiv.org/pdf/1408.5882.pdf) applied CNNs to problems in Natural Language Processing and got good results. In this tutorial, we will introduce how to implement CNN for NLP classification task. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install -U torch

Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.4.0)


In [3]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


## Import require Python libraries

In [0]:
import torch
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator
import spacy
import en_core_web_sm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from tqdm import tqdm, trange
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix

In [5]:
## Set seed of randomization and working device
manual_seed = 77
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

cuda


## Load dataset

In this tutorial, we use the corpus from the [CL-Aff shared task](https://sites.google.com/view/affcon2019/cl-aff-shared-task?authuser=0). HappyDB is a dataset of about 100,000 `happy moments` crowd-sourced via Amazon’s Mechanical Turk where each worker was asked to describe in a complete sentence `what made them happy in the past 24 hours`. Each user was asked to describe three such moments. 
In this tutorial, we focus on `sociality classification`. Sociality refers to `whether or not other people than the author are involved in the emotion situation`. For example, an emotion experience with a sociality value "yes" (i.e., other people are involved) could teach us about social groups (e.g., families) and the range of emotions expressed during specific types of situations (e.g., wedding, death). 

We only use labelled dataset which include 10,560 labelled samples. 

We have already preprocessed (tokenization, removing URLs, mentions, hashtags and so on) the tweets and placed it under ``./happy_db`` folder in three files as ``train.tsv``, ``dev.tsv`` and ``test.tsv``. We split the labeled data into 80\% training set (8,448 moments) and 20\% development set (2112 moments).

Tokenizer

In [0]:
spacy_en = en_core_web_sm.load()
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

`Fileds`

In [0]:
TEXT = Field(sequential=True, tokenize=tokenize_en, lower=True)
LABEL = Field(sequential=False, unk_token = None)

Load dataset using `TabularDataset`

In [0]:
# your code goes here 
train, val, test = TabularDataset.splits(
               path="./drive/My Drive/585_covid19/", # the root directory where the data lies
               train='COVID_500_train.csv', validation="COVID_200_dev.csv", test="COVID_300_test.csv", # file names
               format='csv',
               skip_header=True, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=[('preprocessed_tweets', TEXT), ('label', LABEL)])

Build your vocabulary to map words and labels to integers. 

In [0]:
TEXT.build_vocab(train, min_freq=2)
LABEL.build_vocab(train)

In [11]:
print("Vocabulary size of TEXT:",len(TEXT.vocab.stoi))
print("Vocabulary size of LABEL:",len(LABEL.vocab.stoi))

Vocabulary size of TEXT: 839
Vocabulary size of LABEL: 2


Construct the Iterators to get the train, dev, and test splits. Use `BucketIterator` to initialize the Iterators for the train, dev, and test data.

In [0]:
train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(64,256,256),
 sort_key=lambda x: len(x.preprocessed_tweets), 
 sort=True,
# A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=True
)

In [16]:
# create a single batch and terminate the loop
for batch in train_iter:
    tweets = batch.preprocessed_tweets
    labels = batch.label
    break  #we use first batch as an example.
    
print('tweets:', tweets.shape)
print('labels:', labels.shape)

# tweets: [length, batch size]
# labels: [batch size]

tweets: torch.Size([9, 64])
labels: torch.Size([64])


## Build a `class` for CNN of text classification

In [0]:
# To define a CNN class
class CNN_Text(nn.Module):
    def __init__(self, vocabulary_size, embedding_dim, output_size, kernel_num, region_sizes, dropout):
        '''
        vocabulary_size: vocabulary size
        embedding_dim: word embedding size
        output_size: number of classes in prediction
        kernel_num: number of kernels (number of output channels of convolutional layers)
        region_sizes: height of kernels of convolutional layers
        dropout: dropout rate
        '''
        super(CNN_Text, self).__init__()
        # the size of input channel is 1.
        Ci = 1
        
        # word embedding layer
        self.embeddings = nn.Embedding(num_embeddings = vocabulary_size, embedding_dim = embedding_dim )
        
        # convolution with kernels
        self.convolution_layers = nn.ModuleList([nn.Conv2d(in_channels = Ci, out_channels = kernel_num, kernel_size = (K, embedding_dim)) for K in region_sizes])
        
        # a dropout layer
        self.dropout = nn.Dropout(dropout) 
        
        # fully connected layer
        self.fc = nn.Linear(len(kernel_sizes) * kernel_num, output_size)

    def forward(self, x):
        # input x  [sequence length, batch size]
        
        input_embeddings = self.embeddings(x)  
        # (batch size, word_sequence, embedding_dim) word embedding

        input_embeddings = input_embeddings.permute(1,0,2)
        input_embeddings = input_embeddings.unsqueeze(1)
        #  [batch size, number of channel is one, sequence length, embeeding size]

        # convolutional layers
        convolute_outputs = [F.relu(conv(input_embeddings)).squeeze(3) for conv in self.convolution_layers]  
        
        # to get the maximum value of filtered tensor
        max_pooling_outputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in convolute_outputs] 
        
        concat_list = torch.cat(max_pooling_outputs, 1) # concatenate representations
        
        drop_output = self.dropout(concat_list)  # add drop layer
        
        fc1_output = self.fc(drop_output)  # get the fc1 using a fully connected layer
        
        final_output = F.softmax(fc1_output,dim=1)
        
        return final_output


In [0]:
# Hyper Parameters

# the vocabulary size
vocabulary_size = len(TEXT.vocab.stoi) 

# Dimension of word embedding is 300. Namely, each word is expressed by a vector that has 300 dimensions.
embedding_dim = 300 

# region size as 2, 3, and 4
kernel_sizes = [2,3,4] 

# the number of kernel in each region size
kernels_num = 32  

# The dropout rate is set to be 0.5.
dropout = 0.5

# The output size of labels.
output_size = 2

# learning rate is set to be 0.01.
lr = 0.01        

# The number of iteration is set to be 5.
num_epoch = 5  

# employ class CNN_Text and assign to cnn
model = CNN_Text(vocabulary_size, embedding_dim, output_size, kernels_num, kernel_sizes, dropout).to(device)

criterion = nn.CrossEntropyLoss()

In [19]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.1)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

CNN_Text(
  (embeddings): Embedding(891, 300)
  (convolution_layers): ModuleList(
    (0): Conv2d(1, 32, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 32, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 32, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=96, out_features=2, bias=True)
)

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 353,990 trainable parameters


In [0]:
# Loss and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # define a optimizer for backpropagation
loss_func = nn.CrossEntropyLoss()   # define loss funtion

`train()` and `evaluate()` functions

In [0]:
def train(model, iterator, optimizer, criterion):
    
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        batch_input, labels = batch.preprocessed_tweets, batch.label
        batch_input = batch_input.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        
        outputs = model(batch_input)

        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.cpu().item()

    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    all_pred=[]
    all_label = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            batch_input, labels = batch.preprocessed_tweets, batch.label
            batch_input = batch_input.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(batch_input)

            loss = criterion(outputs, labels)

            epoch_loss += loss.cpu().item()

            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(labels.cpu())
    
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return epoch_loss / len(iterator), accuracy, f1score

In [27]:
# Train the model
MAX_EPOCH = 15
total_step = len(train_iter)
loss_list = []
acc_list = []

for epoch in trange(MAX_EPOCH, desc="Epoch"):
    train_loss = train(model, train_iter, optimizer, criterion)  
    val_loss, val_acc, val_f1 = evaluate(model, val_iter, criterion)

    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    torch.save(state, "./drive/My Drive/585_covid19/ckpt_cnn/CNN_TEXT_"+str(epoch+1)+".pt")

    print('\n Epoch [{}/{}], Train Loss: {:.4f}, Validation Loss: {:.4f}, Validation Accuracy: {:.4f}, Validation F1: {:.4f}'.format(epoch+1, MAX_EPOCH, train_loss, val_loss, val_acc, val_f1))
    

Epoch:  20%|██        | 3/15 [00:00<00:04,  2.44it/s]


 Epoch [1/15], Train Loss: 0.5541, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [2/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [3/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709


Epoch:  33%|███▎      | 5/15 [00:00<00:03,  3.19it/s]


 Epoch [4/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [5/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [6/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709


Epoch:  60%|██████    | 9/15 [00:01<00:01,  5.06it/s]


 Epoch [7/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [8/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [9/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709


Epoch:  73%|███████▎  | 11/15 [00:01<00:00,  6.03it/s]


 Epoch [10/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [11/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [12/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709


Epoch: 100%|██████████| 15/15 [00:01<00:00,  8.48it/s]


 Epoch [13/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [14/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709

 Epoch [15/15], Train Loss: 0.4787, Validation Loss: 0.4233, Validation Accuracy: 0.8900, Validation F1: 0.4709


In [29]:
model_best = CNN_Text(vocabulary_size, embedding_dim, output_size, kernels_num, kernel_sizes, dropout).to(device)
model_best.load_state_dict(torch.load('./drive/My Drive/585_covid19/ckpt_cnn/CNN_TEXT_15.pt')['state_dict'])
model_best = model_best.to(device)
print(evaluate(model_best, test_iter, criterion))

(1.0561595857143402, 0.4066666666666667, 0.2890995260663507)


### Reference
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://medium.com/jatana/report-on-text-classification-using-cnn-rnn-han-f0e887214d5f
* http://www.davidsbatista.net/blog/2018/03/31/SentenceClassificationConvNets/